In [157]:
%matplotlib inline
import numpy as np
import pandas as pd
import os
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn import linear_model
from sklearn import naive_bayes

In [128]:
imdb_dir = '/home/viviane/Documentos/datasets/imdb/aclImdb/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')
labels = []
texts = []
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [129]:
test_dir = os.path.join(imdb_dir, 'test')
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(test_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

### Questão 01

In [131]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

In [132]:
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [133]:
stop = stopwords.words('english')

In [134]:
def preprocessing(texto):
    texto = preprocessor(texto)
    aux = [w for w in tokenizer_porter(texto) if w not in stop]
    output = ''
    output = output + aux[0]
    for i in aux[1:]:
        output = output + ' ' + i
    return output

In [136]:
for i in range(0,len(texts)):
    texts[i] = preprocessing(texts[i])

In [135]:
texts[0]

'Just another example of why Stepehn King\'s books should not be made into movies. (Even Carrie, one of the best, is ruined in the adaptation from book to screen.) The premise of the story revolves around a fat lawyer, always on a diet, who "accidentally" kills an old gypsy woman. In court, with the help of the judge and the local police chief, he gets off, even though the accident was sort of his fault as he was not paying attention as he was driving. The father of the dead gypsy woman places a curse on the 3, with our main character, Billy (the lawyer) getting thinner and thinner by the day. Though the movie kept with the book, for the most part, and has your typical King ending, the acting was stilted and felt forced. We went from one scene to the next without much of anything in between, sort of like reading acts in a play. King himself made a cameo in the movie (sort of like Dave Barry), which reinforces my belief that authors should stay just that: authors. Leave the acting to ac

In [137]:
texts[0]

'anoth exampl whi stepehn king book made movi even carri one best ruin adapt book screen premis stori revolv around fat lawyer alway diet accident kill old gypsi woman court help judg local polic chief get even though accid wa sort hi fault wa pay attent wa drive father dead gypsi woman place curs 3 main charact billi lawyer get thinner thinner day though movi kept book part ha typic king end act wa stilt felt forc went one scene next without much anyth sort like read act play king made cameo movi sort like dave barri reinforc belief author stay author leav act actor anyon thi movi wa great seen major charact movi much better'

### Questão 02

In [138]:
X_train = texts[:45000]
y_train = labels[:45000]
X_test = texts[45000:]
y_test = labels[45000:]

In [139]:
count_vect = CountVectorizer()

In [140]:
X_train_counts = count_vect.fit_transform(X_train)
X_test_counts = count_vect.fit_transform(X_test)

In [141]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

In [142]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_test_counts)
X_test_tf = tf_transformer.transform(X_test_counts)

In [144]:
clf = MultinomialNB().fit(X_train_tf, y_train)

In [146]:
y_pred = clf.predict(X_train_tf[40000:])

In [148]:
metrics.accuracy_score(y_train[40000:],y_pred)

0.5888

### Questão 03

In [149]:
vectorizer = HashingVectorizer(n_features=2**21)

In [150]:
X = vectorizer.fit_transform(X_train)

In [153]:
model = linear_model.SGDClassifier()

In [154]:
model.fit(X[:40000],y_train[:40000])

/home/viviane/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [155]:
y_pred = model.predict(X[40000:])

In [156]:
metrics.accuracy_score(y_train[40000:],y_pred)

0.7948

### Questão 04

In [158]:
model_gaussian = naive_bayes.GaussianNB()
model_multinomial = naive_bayes.MultinomialNB()
model_bernoulli = naive_bayes.BernoulliNB()

In [162]:
# model_gaussian.fit(X[:40000],y_train[:40000])
# model_multinomial.fit(X[:40000],y_train[:40000])
model_bernoulli.fit(X[:40000],y_train[:40000])

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [163]:
y_pred_bernoulli = model.predict(X[40000:])

In [164]:
metrics.accuracy_score(y_train[40000:],y_pred_bernoulli)

0.7948